# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355115467275                   -1.10    6.1    159ms
  2   -9.356756685130       -2.78       -1.43    1.0   81.3ms
  3   -9.357074787756       -3.50       -2.80    2.4    130ms
  4   -9.357119580708       -4.35       -3.00    7.6    277ms
  5   -9.357119998203       -6.38       -3.15    1.0    356ms
  6   -9.357120165069       -6.78       -3.30    1.0   83.3ms
  7   -9.357120262169       -7.01       -3.47    2.2   98.3ms
  8   -9.357120297942       -7.45       -3.63    1.0   81.9ms
  9   -9.357120317551       -7.71       -3.90    1.0   81.5ms
 10   -9.357120322311       -8.32       -4.15    1.1    106ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74304793555                   -0.97    6.2    478ms
  2   -18.74989488621       -2.16       -1.36    1.0    249ms
  3   -18.77979396900       -1.52       -1.70    5.4    382ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      4
│     12
│      1
│      1
│      1
│      6
│      1
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  4   -18.79241139415       -1.90       -1.72    3.5    344ms
  5   -18.79222728416   +   -3.73       -1.72    1.8    284ms
  6   -18.79095053339   +   -2.89       -1.64    3.0    306ms
  7   -18.78212879568   +   -2.05       -1.24    1.5    263ms
  8   -18.76062820054   +   -1.67       -0.99    2.1    311ms
  9   -18.75503522672   +   -2.25       -0.96    2.1    284ms
 10   -18.75007423124   +   -2.30       -0.93    1.0    242ms
 11   -18.75669420919       -2.18    

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54209809260                   -0.84    9.5    1.98s
  2   -37.55618310102       -1.85       -1.21    3.1    895ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     14
│      5
│     18
│     15
│     16
│     17
│      9
│      9
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -37.56454362689       -2.08       -2.11   12.9    1.74s
  4   -37.56493686907       -3.41       -2.37    9.1    1.66s
  5   -37.56497937498       -4.37       -3.27    3.5    1.28s
  6   -37.56498443622       -5.30       -3.48    8.2    1.60s
  7   -37.56498515217       -6.15       -4.15    3.1    876ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355232587472                   -1.10    5.9    172ms
  2   -9.356793745173       -2.81       -1.90    1.0   64.2ms
  3   -9.357021610694       -3.64       -2.26    6.0    151ms
  4   -9.357113014807       -4.04       -2.81    2.5   87.4ms
  5   -9.357120081928       -5.15       -3.71    3.2   98.8ms
  6   -9.357120275567       -6.71       -3.95    4.6    133ms
  7   -9.357120308033       -7.49       -4.15    1.5   81.2ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.53881194750                   -0.84    8.8    1.92s
  2   -37.55162470053       -1.89       -1.47    2.8    711ms
  3   -36.73424564636   +   -0.09       -0.97   12.9    1.80s
  4   -28.45205392454   +    0.92       -0.59    6.8    1.89s
  5   -36.76733350620        0.92       -1.07    7.2    1.69s
  6   -37.55075267035       -0.11       -1.90    4.4    1.23s
  7   -37.53782012362   +   -1.89       -1.82    4.2    1.25s
  8   -37.46618002550   +   -1.14       -1.56    5.2    1.56s
  9   -37.55771720517       -1.04       -1.96    4.2    991ms
 10   -37.56426150891       -2.18       -2.36    1.8    768ms
 11   -37.56472576576       -3.33       -2.62    3.1    863ms
 12   -37.56484705282       -3.92       -2.81    2.9    901ms
 13   -37.56492905879       -4.09       -3.07    2.6    1.15s
 14   -37.56496811358       -4.41       -3.34    2.9    879ms
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.